In [1]:
import pandas as pd
import os
from pathlib import Path

# Define o caminho da pasta
logs_path = Path('logs_em_tratamento')

# Lista todos os arquivos de bloco concatenados
bloco_files = list(logs_path.glob('bloco_*_concatenado.csv'))

# Cria uma lista para armazenar os DataFrames
dfs = []

# Lê cada arquivo mantendo a estrutura original
for file in bloco_files:
    df = pd.read_csv(file, sep='\t')  # Remove a definição manual de colunas
    dfs.append(df)

# Concatena todos os DataFrames
logs_totais_tratados = pd.concat(dfs, ignore_index=True)

# Remove duplicatas se houver
#logs_totais_tratados = logs_totais_tratados.drop_duplicates()

# Salva o DataFrame consolidado mantendo o formato original
output_file = logs_path / 'logs_totais_tratados.csv'
logs_totais_tratados.to_csv(output_file, sep='\t', index=False)  # Adiciona sep='\t'

print(f"Arquivo criado com sucesso em: {output_file}")
print(f"Total de registros: {len(logs_totais_tratados)}")

Arquivo criado com sucesso em: logs_em_tratamento\logs_totais_tratados.csv
Total de registros: 13485


In [2]:
# Lê o arquivo original
df = pd.read_csv('logs_em_tratamento/logs_totais_tratados.csv', sep='\t')

# Separa por gênero
df_masculino = df[df['GeneroCod'] == 'm']
df_feminino = df[df['GeneroCod'] == 'f']

# Salva os arquivos separados
output_masculino = logs_path / 'logs_masculino.csv'
output_feminino = logs_path / 'logs_feminino.csv'

df_masculino.to_csv(output_masculino, sep='\t', index=False)
df_feminino.to_csv(output_feminino, sep='\t', index=False)

print(f"Dataset masculino salvo em: {output_masculino}")
print(f"Total de registros masculinos: {len(df_masculino)}")
print(f"\nDataset feminino salvo em: {output_feminino}")
print(f"Total de registros femininos: {len(df_feminino)}")

Dataset masculino salvo em: logs_em_tratamento\logs_masculino.csv
Total de registros masculinos: 7025

Dataset feminino salvo em: logs_em_tratamento\logs_feminino.csv
Total de registros femininos: 6460


In [3]:
def get_majority_class(group):
    # Conta as ocorrências de cada classificação
    value_counts = group.value_counts()
    
    # Se houver empate entre as classificações mais frequentes
    if len(value_counts) > 1 and value_counts.iloc[0] == value_counts.iloc[1]:
        return 'empate'
    
    # Retorna a classificação mais frequente
    return value_counts.index[0]

def get_totals(group):
    # Conta ocorrências de cada tipo de classificação
    counts = group.value_counts()
    
    # Retorna os totais em um dicionário
    return {
        'classificacao_majoritaria': get_majority_class(group),
        'total_positiva': counts.get('positiva', 0),
        'total_negativa': counts.get('negativa', 0),
        'total_neutra': counts.get('neutra', 0)
    }

# Processa dataset masculino
agg_masculino = (df_masculino
                .groupby('frase')
                .agg({
                    'Value': get_totals,
                    'duracao': 'mean',
                    'ParticipantMD5': 'count'
                }))

# Expande os resultados da agregação Value
value_columns = pd.DataFrame(agg_masculino['Value'].tolist(), index=agg_masculino.index)
agg_masculino = agg_masculino.drop('Value', axis=1)
agg_masculino = pd.concat([agg_masculino, value_columns], axis=1)

# Renomeia as colunas
agg_masculino = agg_masculino.rename(columns={
    'duracao': 'duracao_media',
    'ParticipantMD5': 'total_classificacoes'
})

# Reset do índice
agg_masculino = agg_masculino.reset_index()

# Processa dataset feminino (mesmo processo)
agg_feminino = (df_feminino
               .groupby('frase')
               .agg({
                   'Value': get_totals,
                   'duracao': 'mean',
                   'ParticipantMD5': 'count'
               }))

# Expande os resultados da agregação Value
value_columns = pd.DataFrame(agg_feminino['Value'].tolist(), index=agg_feminino.index)
agg_feminino = agg_feminino.drop('Value', axis=1)
agg_feminino = pd.concat([agg_feminino, value_columns], axis=1)

# Renomeia as colunas
agg_feminino = agg_feminino.rename(columns={
    'duracao': 'duracao_media',
    'ParticipantMD5': 'total_classificacoes'
})

# Reset do índice
agg_feminino = agg_feminino.reset_index()

# Salva os datasets
agg_masculino.to_csv(logs_path / 'logs_masculino_agregado.csv', sep='\t', index=False)
agg_feminino.to_csv(logs_path / 'logs_feminino_agregado.csv', sep='\t', index=False)

# Exibe informações
print("Datasets agregados criados com sucesso!")
print(f"Total de frases únicas (masculino): {len(agg_masculino)}")
print(f"Total de frases únicas (feminino): {len(agg_feminino)}")

# Exibe as primeiras linhas
pd.set_option('display.max_columns', None)
print("\nPrimeiras linhas do dataset masculino agregado:")
print(agg_masculino.head())
print("\nPrimeiras linhas do dataset feminino agregado:")
print(agg_feminino.head())

Datasets agregados criados com sucesso!
Total de frases únicas (masculino): 1463
Total de frases únicas (feminino): 1463

Primeiras linhas do dataset masculino agregado:
                                               frase  duracao_media  \
0   A história fala de três amigas gordinhas, que...        8.46725   
1   Ainda mais que tudo que tá acontecendo ultima...        5.56780   
2   Amo você mais do que a mim mesma, afinal você...        2.92420   
3   Ela fica jogando na minha cara que eu não sei...        8.11675   
4   Ele sempre esteve do meu lado e o que eu mais...        8.63960   

   total_classificacoes classificacao_majoritaria  total_positiva  \
0                     4                  negativa               1   
1                     5                    neutra               1   
2                     5                  positiva               5   
3                     4                  negativa               0   
4                     5                  positiva         

In [4]:
# Identifica frases com empate em ambos os datasets
empates_df = pd.merge(
    agg_masculino[agg_masculino['classificacao_majoritaria'] == 'empate'][['frase']],
    agg_feminino[agg_feminino['classificacao_majoritaria'] == 'empate'][['frase']],
    on='frase',
    how='inner'
)

# Adiciona as informações completas de ambos os datasets
empates_df = empates_df.merge(
    agg_masculino, 
    on='frase', 
    suffixes=('', '_m')
).merge(
    agg_feminino, 
    on='frase', 
    suffixes=('_m', '_f')
)

# Salva o dataset de empates
output_empates = logs_path / 'empates_duplos.csv'
empates_df.to_csv(output_empates, sep='\t', index=False)

print(f"Dataset de empates duplos criado em: {output_empates}")
print(f"Total de frases com empate em ambos os gêneros: {len(empates_df)}")
print("\nPrimeiras linhas do dataset de empates:")
print(empates_df.head())

Dataset de empates duplos criado em: logs_em_tratamento\empates_duplos.csv
Total de frases com empate em ambos os gêneros: 26

Primeiras linhas do dataset de empates:
                                               frase  duracao_media_m  \
0  A verdade é que sou intensa demais e não há qu...         4.569750   
1  Achei uma solução, ou melhor, na minha cabeça ...         4.007750   
2  Afinal, para mim, que sempre levei vida de esp...        43.402250   
3  Ao cair da noite nos dirigimos ao hotel mais p...         5.843500   
4             Brincar de comidinha com as vizinhas !         2.559833   

   total_classificacoes_m classificacao_majoritaria_m  total_positiva_m  \
0                       4                      empate                 2   
1                       4                      empate                 2   
2                       4                      empate                 2   
3                       4                      empate                 2   
4                  